In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch
from torch import nn

In [ ]:
! pip3 install pyro-ppl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
import pyro

In [ ]:
data_path = "/content/drive/MyDrive/Projects/Capstone/Clean Energy Dev Tool/Data/CAISO_data.csv"

In [ ]:
data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data = data.sort_values('timestamp')
data['timestamp'] = data['timestamp'].astype(np.datetime64).dt.tz_localize('UTC').dt.tz_convert('US/Pacific') # Convert times to pacific
data

,timestamp,freq,market,ba_name,load_MW
171,2015-01-01 00:00:00-08:00,5m,RT5M,CAISO,23041.00
55,2015-01-01 00:05:00-08:00,5m,RT5M,CAISO,22937.00
77,2015-01-01 00:10:00-08:00,5m,RT5M,CAISO,22824.00
132,2015-01-01 00:15:00-08:00,5m,RT5M,CAISO,22707.00
111,2015-01-01 00:20:00-08:00,5m,RT5M,CAISO,22596.00
...,...,...,...,...,...
838049,2022-12-31 23:35:00-08:00,5m,RT5M,CAISO,20993.08
838194,2022-12-31 23:40:00-08:00,5m,RT5M,CAISO,20941.46
838104,2022-12-31 23:45:00-08:00,5m,RT5M,CAISO,20734.96
838105,2022-12-31 23:50:00-08:00,5m,RT5M,CAISO,20673.72


In [ ]:
df_2022 = data[(data['timestamp'].dt.year == 2022)]
df_2022

,timestamp,freq,market,ba_name,load_MW
733704,2022-01-01 00:00:00-08:00,5m,RT5M,CAISO,22002.18
733762,2022-01-01 00:05:00-08:00,5m,RT5M,CAISO,21925.60
733789,2022-01-01 00:10:00-08:00,5m,RT5M,CAISO,21846.55
733731,2022-01-01 00:15:00-08:00,5m,RT5M,CAISO,21766.07
733763,2022-01-01 00:20:00-08:00,5m,RT5M,CAISO,21707.94
...,...,...,...,...,...
838049,2022-12-31 23:35:00-08:00,5m,RT5M,CAISO,20993.08
838194,2022-12-31 23:40:00-08:00,5m,RT5M,CAISO,20941.46
838104,2022-12-31 23:45:00-08:00,5m,RT5M,CAISO,20734.96
838105,2022-12-31 23:50:00-08:00,5m,RT5M,CAISO,20673.72


# Building Empirical Priors

Let's take a stab at building some empirical priors. This involves the following steps:
1. Each day gets a duck curve.
  - That means we need to process our data and split them into days.
  - Some random set of days should be set aside as test days.
2. For each duck curve, we fit a model. For now, we'll try a PyTorch MLP. The model should be able to achieve close to 100% accuracy, but shouldn't be overparameterized.
  - We can either use MSE or log-likelihood maximization, since both of these will give us similar answers.

## Data Processing

First, we need to do some feature engineering for our data. In particular, our model needs to receive information about:
1. Time of day
2. Whether the day is a weekend or weekday

In [ ]:
# Minutes since beginning of day
df_2022['minutes'] = df_2022['timestamp'].dt.hour * 60 + df_2022['timestamp'].dt.minute
df_2022

<ipython-input-8-6ab6f6f639f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['minutes'] = df_2022['timestamp'].dt.hour * 60 + df_2022['timestamp'].dt.minute


,timestamp,freq,market,ba_name,load_MW,minutes
733704,2022-01-01 00:00:00-08:00,5m,RT5M,CAISO,22002.18,0
733762,2022-01-01 00:05:00-08:00,5m,RT5M,CAISO,21925.60,5
733789,2022-01-01 00:10:00-08:00,5m,RT5M,CAISO,21846.55,10
733731,2022-01-01 00:15:00-08:00,5m,RT5M,CAISO,21766.07,15
733763,2022-01-01 00:20:00-08:00,5m,RT5M,CAISO,21707.94,20
...,...,...,...,...,...,...
838049,2022-12-31 23:35:00-08:00,5m,RT5M,CAISO,20993.08,1415
838194,2022-12-31 23:40:00-08:00,5m,RT5M,CAISO,20941.46,1420
838104,2022-12-31 23:45:00-08:00,5m,RT5M,CAISO,20734.96,1425
838105,2022-12-31 23:50:00-08:00,5m,RT5M,CAISO,20673.72,1430


In [ ]:
prior_sets = []

curr_day = np.datetime64("2022-01-01")
next_day = np.datetime64('2022-01-02')

while next_day <= np.datetime64('2023-01-01'):
    is_curr_year = df_2022['timestamp'].dt.year == pd.DatetimeIndex([curr_day]).year[0]
    is_curr_month = df_2022['timestamp'].dt.month == pd.DatetimeIndex([curr_day]).month[0]
    is_curr_day = df_2022['timestamp'].dt.day == pd.DatetimeIndex([curr_day]).day[0]
    prior_sets.append(df_2022[is_curr_year & is_curr_month & is_curr_day])
    curr_day += np.timedelta64(1, 'D')
    next_day += np.timedelta64(1, 'D')

In [ ]:
prior_sets[0]

,timestamp,freq,market,ba_name,load_MW,minutes
733704,2022-01-01 00:00:00-08:00,5m,RT5M,CAISO,22002.18,0
733762,2022-01-01 00:05:00-08:00,5m,RT5M,CAISO,21925.60,5
733789,2022-01-01 00:10:00-08:00,5m,RT5M,CAISO,21846.55,10
733731,2022-01-01 00:15:00-08:00,5m,RT5M,CAISO,21766.07,15
733763,2022-01-01 00:20:00-08:00,5m,RT5M,CAISO,21707.94,20
...,...,...,...,...,...,...
733883,2022-01-01 23:35:00-08:00,5m,RT5M,CAISO,22156.79,1415
733825,2022-01-01 23:40:00-08:00,5m,RT5M,CAISO,22062.86,1420
733849,2022-01-01 23:45:00-08:00,5m,RT5M,CAISO,21970.24,1425
733761,2022-01-01 23:50:00-08:00,5m,RT5M,CAISO,21879.17,1430


In [ ]:
# Split data into training and testing
# We will only be using our data in the training set to generate our empirical prior distributions; otherwise we may cause data leakage issues.
shuffled_indices = pd.DataFrame(np.arange(365)).sample(frac=1, replace=False, random_state=101).index.tolist()
test_set = np.array(prior_sets)[shuffled_indices[:30]]
train_set = np.array(prior_sets)[shuffled_indices[30:]]

<ipython-input-11-3fb556538b1c>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_set = np.array(prior_sets)[shuffled_indices[:30]]
<ipython-input-11-3fb556538b1c>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_set = np.array(prior_sets)[shuffled_indices[30:]]


In [ ]:
torch.utils.data.DataLoader(train_set, batch_size=5, shuffle=True)

### Defining our model

In [ ]:
class MultiLayerPerceptron(nn.Module):
  def __init__(self, in_dim, layer_dims, out_dim):
    super().__init__()

    self.n_layers = len(layer_dims) + 1

    self.layer_dims = [in_dim] + layer_dims + [out_dim]
    self.layers = nn.ModuleList(
        [nn.Linear(self.layer_dims[i], self.layer_dims[i+1]) for i in np.arange(self.n_layers)]
    )
    self.activations = nn.ModuleList([nn.ReLU() for i in np.arange(self.n_layers - 1)] + [nn.Identity()])

  def forward(self, x):
    output = x
    for i in np.arange(self.n_layers):
      output = self.layers[i](output)
      output = self.activations[i](output)
    return output.squeeze()

In [ ]:
(np.ones(3) * 5).astype(int).tolist()

[5, 5, 5]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", device)

epochs = 10
batch_size = 10
learning_rate = 0.01

num_features = 1
output_dim = 1
layer_dims = (np.ones(3) * 5).astype(int).tolist() # 2 hidden layers, width 5
model = MultiLayerPerceptron(num_features, layer_dims, output_dim).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
dataloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=True)

all_losses = []
all_valid_losses = []
model.train() # Put model in training mode
for epoch in range(epochs):
    training_losses = []
    valid_losses = []
    for x, y in tqdm.notebook.tqdm(dataloader, unit="batch"):
        x, y = x.float().to(device), y.float().to(device)
        optimizer.zero_grad() # Remove the gradients from the previous step
        pred = model(x.reshape(x.shape[0], x.shape[1], x.shape[2] * x.shape[3]))
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        training_losses.append(loss.item())

    with torch.no_grad():
      model.eval() # Put model in eval mode
      num_correct = 0
      for x, y in valid_dataloader:
          x, y = x.float().to(device), y.float().to(device)
          pred = model(x.reshape(x.shape[0], x.shape[1], x.shape[2] * x.shape[3]))
          loss = criterion(pred, y)
          valid_losses.append(loss.item())
      print("Validation Accuracy:", num_correct / len(validation_data))
      model.train() # Put model back in train mode

    all_losses.append(np.mean(training_losses))
    all_valid_losses.append(np.mean(valid_losses))
    print("Finished Epoch", epoch + 1, ", training loss:", np.mean(training_losses))

In [ ]:
df_2022 = data[data['timestamp'].dt.year == 2022]
df_2022

AttributeError: ignored